<a href="https://colab.research.google.com/github/KdimAlex/Sentiment-Classification-Project/blob/master/Make_Inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Download model that contains token vectors that the tokens map to:
!python -m spacy download en_core_web_md

2023-11-01 20:01:03.330349: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-01 20:01:03.330419: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-01 20:01:03.330464: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-01 20:01:03.343671: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 20:01:05.611754: W tensorflow/compiler/

In [ ]:
import spacy
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Load the spaCy model with GloVe embeddings
nlp = spacy.load("en_core_web_md")

def get_tok_embedding(paragraph):
  tokens = word_tokenize(paragraph)
  tok_embeddings = [nlp(token).vector for token in tokens]
  return tok_embeddings

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np

def get_pos_embedding (paragraph):
  tokens = word_tokenize(paragraph)
  position = np.arange(len(tokens))[:, np.newaxis]
  div_term = np.exp(np.arange(0, 300, 2) * -(np.log(10000.0) / 300))
  pos_embeddings = np.zeros((len(tokens), 300))
  pos_embeddings[:, 0::2] = np.sin(position * div_term)
  pos_embeddings[:, 1::2] = np.cos(position * div_term)
  return pos_embeddings

In [ ]:
#TEMPORARY METHOD TO JUST COMPILE FOR SAM TO FIX
def get_seg_embedding (paragraph):
  tokens = word_tokenize(paragraph)
  seg_embeddings = np.ones((len(tokens), 300))
  return seg_embeddings

In [ ]:
def get_final_embedding (tok_embeddings, pos_embeddings, seg_embeddings):
  # Ensure that all the embeddings have the same shape
  assert len(tok_embeddings) == len(pos_embeddings) == len(seg_embeddings), "All embeddings must have the same length"
  # Convert token embeddings list to numpy array if it's not already
  if isinstance(tok_embeddings, list):
      tok_embeddings = np.array(tok_embeddings)
  # Sum the embeddings element-wise
  pre_self_attention_embeddings = tok_embeddings + pos_embeddings + seg_embeddings
  return pre_self_attention_embeddings

In [ ]:
def scaled_dot_product_attention(query, key, value):
    d_k = query.shape[1]  # dimensionality of the key
    scores = np.matmul(query, key.T) / np.sqrt(d_k)
    weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)
    return np.matmul(weights, value), weights

def get_self_attention_embeddings(embeddings, d_model=300):
    # Initialize weights for the self-attention layers
    # For simplicity, these are random. In practice, they would be learned parameters.
    W_q = np.random.rand(d_model, d_model)
    W_k = np.random.rand(d_model, d_model)
    W_v = np.random.rand(d_model, d_model)

    # Calculate query, key, value matrices
    query = np.matmul(embeddings, W_q)
    key = np.matmul(embeddings, W_k)
    value = np.matmul(embeddings, W_v)

    # Compute scaled dot-product attention
    attention_output, _ = scaled_dot_product_attention(query, key, value)

    return attention_output

In [ ]:
paragraph = "My name is Tom, and I am unbelievably hungry."

token_embeddings = get_tok_embedding (paragraph)
position_embeddings = get_pos_embedding (paragraph)
segment_embeddings = get_seg_embedding (paragraph)
final_embedding = get_final_embedding (token_embeddings, position_embeddings, segment_embeddings)

transformer_input = get_self_attention_embeddings(final_embedding)

<ipython-input-37-ccc21ca15832>:4: RuntimeWarning: overflow encountered in exp
  weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)
<ipython-input-37-ccc21ca15832>:4: RuntimeWarning: invalid value encountered in divide
  weights = np.exp(scores) / np.sum(np.exp(scores), axis=-1, keepdims=True)


In [ ]:
# Tester for while I am coding:
def print_embeddings(tokens, embeddings, embedding_type, num_values=10, decimal_places=4):
    for token, embedding in zip(tokens, embeddings):
        limited_embedding = embedding[:num_values]
        formatted_embedding = [round(value, decimal_places) for value in limited_embedding]
        print(f"{embedding_type} for Token: {token}")
        print(f"{embedding_type} Embedding (first {num_values} values of {len(embedding)} values): {formatted_embedding}\n")

tokens = word_tokenize(paragraph)
print_embeddings(tokens, token_embeddings, "Token")
print_embeddings(tokens, position_embeddings, "Position")
print_embeddings(tokens, segment_embeddings, "Segment")
print_embeddings(tokens, final_embedding, "Summed Final")


Token for Token: My
Token Embedding (first 10 values of 300 values): [7.6432, 1.5701, -11.373, -7.1437, 3.5173, 1.6061, 1.5734, -0.1969, -3.7184, 5.7551]

Token for Token: name
Token Embedding (first 10 values of 300 values): [0.9176, -0.7391, 8.3935, -3.5195, 7.3546, -1.2438, 1.3083, -5.036, 2.619, 0.1279]

Token for Token: is
Token Embedding (first 10 values of 300 values): [1.475, 6.0078, 1.1205, -3.5874, 3.7638, 3.1987, -2.206, 3.2128, -2.0816, -0.0029]

Token for Token: Tom
Token Embedding (first 10 values of 300 values): [-2.8594, -1.1686, 0.1836, -5.2091, 0.6395, 4.5149, 3.3691, -0.0328, -1.1249, -0.5381]

Token for Token: ,
Token Embedding (first 10 values of 300 values): [-3.3899, -4.7034, -0.561, 1.2291, 4.3298, -1.0775, -1.3006, 8.7939, -0.1667, -4.3738]

Token for Token: and
Token Embedding (first 10 values of 300 values): [-3.3477, -6.0854, -3.6366, 0.5348, 8.4195, 1.3784, 0.3117, 5.6086, -0.7768, -3.3725]

Token for Token: I
Token Embedding (first 10 values of 300 values)